# Study of Image classification with modern MLP Mixer model and CKA

**Author:** [Arturo Flores](https://www.linkedin.com/in/afloresalv/)<br>
**Based on (MLP-MIXER):**  https://keras.io/examples/vision/mlp_image_classification/

# Setup for the MLP-Mixer Architecture

################################################################

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
import matplotlib.pyplot as plt 
from scipy.stats import norm
from matplotlib.ticker import FormatStrFormatter
import datetime
import pickle
# Files imported from the sleected GitHub https://cka-similarity.github.io/
from CKA_Google import *
import seaborn as sns 
import random
import matplotlib.pyplot as plt 

C:\Users\alach\AppData\Roaming\Python\Python38\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.5.0 and strictly below 2.8.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


# Experiment 1 : Understand Network Architecture

## Configure the hyperparameters

In [2]:
weight_decay = 0.0001
batch_size = 512 
num_epochs = 50
dropout_rate = 0.2
learning_rate = 0.005

## Selected Architecture: B/32

image_size = 224  # We'll resize input images to this size. Square
patch_size = 32  # Size of the patches to be extracted from the input images. Square
num_patches = (image_size // patch_size) ** 2  # Size of the data array, or sequence length (S)
embedding_dim = [384]  # Fixed Embedding Dimension
num_blocks = [8,12,24,32]

print(f"Image size: {image_size} X {image_size} = {image_size ** 2}")
print(f"Patch size: {patch_size} X {patch_size} = {patch_size ** 2} ")
print(f"Patches per image: {num_patches}")
print(f"Elements per patch (3 channels): {(patch_size ** 2) * 3}")

now = datetime.datetime.now()
date = now.strftime("%Y-%m-%d_%H-%M")


Image size: 224 X 224 = 50176
Patch size: 32 X 32 = 1024 
Patches per image: 49
Elements per patch (3 channels): 3072


## Prepare the data

In [3]:
#Dataset for training 

num_classes = 10
input_shape = (32, 32, 3)

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")
#plt.imshow(x_train[1])

x_train shape: (50000, 32, 32, 3) - y_train shape: (50000, 1)
x_test shape: (10000, 32, 32, 3) - y_test shape: (10000, 1)


## Build a classification model

In [4]:
def build_classifier(blocks, embedding_dim, positional_encoding=False):
    inputs = layers.Input(shape=input_shape)
    # Augment data. 
    augmented = data_augmentation(inputs)
    # Create patches. 
    patches = Patches(patch_size, num_patches)(augmented)
    # Encode patches to generate a [batch_size, num_patches, embedding_dim] tensor.
    x = layers.Dense(units=embedding_dim)(patches)
    if positional_encoding:
        positions = tf.range(start=0, limit=num_patches, delta=1)
        position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=embedding_dim
        )(positions)
        x = x + position_embedding
    # Process x using the module blocks. ## (sequential_82)
    x = blocks(x)
    # Apply global average pooling to generate a [batch_size, embedding_dim] representation tensor. 
    representation = layers.GlobalAveragePooling1D()(x)
    # Apply dropout.
    representation = layers.Dropout(rate=dropout_rate)(representation)
    # Compute logits outputs.
    logits = layers.Dense(num_classes)(representation) 
    # Create the Keras model.
    return keras.Model(inputs=inputs, outputs=logits)


## Define an experiment

In [5]:
def run_experiment(model):
    # Create Adam optimizer with weight decay. Regularization that penalizes the increase of weight - with a facto alpha - to correct the overfitting
    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay,
    )
    # Compile the model.
    model.compile(
        optimizer=optimizer,
        #Negative Log Likelihood = Categorical Cross Entropy
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="acc"),
            keras.metrics.SparseTopKCategoricalAccuracy(5, name="top5-acc"),
        ],
    )
    # Create a learning rate scheduler callback.
    reduce_lr = keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=5
    )
    # Create an early stopping regularization callback. 
    # It ends at a point that corresponds to a minimum of the L2-regularized objective
    #early_stopping = tf.keras.callbacks.EarlyStopping(
    #    monitor="val_loss", patience=10, restore_best_weights=True
    #)
    # Fit the model.
    history = model.fit(
        x=x_train,
        y=y_train,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_split=0.1,
        callbacks=[reduce_lr],
    )

    _, accuracy, top_5_accuracy = model.evaluate(x_test, y_test)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

    # Return history to plot learning curves.
    return history, accuracy, top_5_accuracy

## Use data augmentation
Their state is not set during training; it must be set before training, either by initializing them from a precomputed constant, or by "adapting" them on data.

In [6]:
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.Resizing(image_size, image_size),
        layers.RandomFlip("horizontal"),
        layers.RandomZoom(
            height_factor=0.2, width_factor=0.2
        ),
    ],
    name="data_augmentation",
)
# Compute the mean and the variance of the training data for normalization.
data_augmentation.layers[0].adapt(x_train)


## Implement patch extraction as a layer

In [7]:

class Patches(layers.Layer):
    def __init__(self, patch_size, num_patches):
        super(Patches, self).__init__()
        self.patch_size = patch_size
        self.num_patches = num_patches

    def call(self, images):
        #Extract the shape dimension in the position 0 = columns
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            #Without overlapping, stride horizontally and vertically
            strides=[1, self.patch_size, self.patch_size, 1],
            #Rate: Dilation factor [1 1* 1* 1] controls the spacing between the kernel points.
            rates=[1, 1, 1, 1],
            #Patches contained in the images are considered, no zero padding
            padding="VALID",
        )
        #shape[-1], number of colummns, as well as shape[0]
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, self.num_patches, patch_dims])
        return patches

    def get_config(self):
        config = super(Patches, self).get_config().copy()
        config.update ({
            'patch_size' : self.patch_size ,
            'num_patches' : self.num_patches
        })
        return config

## The MLP-Mixer model

The MLP-Mixer is an architecture based exclusively on
multi-layer perceptrons (MLPs), that contains two types of MLP layers:

1. One applied independently to image patches, which mixes the per-location features.
2. The other applied across patches (along channels), which mixes spatial information.

This is similar to a [depthwise separable convolution based model](https://arxiv.org/pdf/1610.02357.pdf)
such as the Xception model, but with two chained dense transforms, no max pooling, and layer normalization
instead of batch normalization.

### Implement the MLP-Mixer module

In [8]:

class MLPMixerLayer(layers.Layer):
    def __init__(self, num_patches, embedding_dim, dropout_rate, *args, **kwargs):
        super(MLPMixerLayer, self).__init__(*args, **kwargs)

        self.mlp1 = keras.Sequential(
            [
                layers.Dense(units=num_patches),
                tfa.layers.GELU(),
                layers.Dense(units=num_patches),
                layers.Dropout(rate=dropout_rate),
            ]
        )

        self.mlp2 = keras.Sequential(
            [
                layers.Dense(units=num_patches),
                tfa.layers.GELU(),
                layers.Dense(units=embedding_dim),
                layers.Dropout(rate=dropout_rate),
            ]
        )
        self.normalize = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs):
        # Apply layer normalization.
        x = self.normalize(inputs)
        # Transpose inputs from [num_batches, num_patches, hidden_units] to [num_batches, hidden_units, num_patches].
        x_channels = tf.linalg.matrix_transpose(x)
        # Apply mlp1 on each channel independently.
        mlp1_outputs = self.mlp1(x_channels)
        # Transpose mlp1_outputs from [num_batches, hidden_dim, num_patches] to [num_batches, num_patches, hidden_units].
        mlp1_outputs = tf.linalg.matrix_transpose(mlp1_outputs)
        # Add skip connection.
        x = mlp1_outputs + inputs
        # Apply layer normalization.
        x_patches = self.normalize(x)
        # Apply mlp2 on each patch independtenly.
        mlp2_outputs = self.mlp2(x_patches)
        # Add skip connection.
        x = x + mlp2_outputs
        return x

    def get_config(self): 
        config = super(MLPMixerLayer, self).get_config().copy()
        config.update ({
            'num_patches' : num_patches,
            'embedding_dim' : embedding_dim,
            'dropout_rate' : dropout_rate,
        })
        return config

## Build, train, and evaluate the MLP-Mixer model

Note that training the model with the current settings on a V100 GPUs
takes around 8 seconds per epoch.

In [9]:
# Report: Learning Curve
def curves(history):
    ymax1 = min(history["loss"])
    xmax1 = history["loss"].index(ymax1)
    ymax2 = min(history["val_loss"])
    xmax2 = history["val_loss"].index(ymax2)
    plt.title("Cross Entropy Loss")
    plt.plot(history["loss"], color = 'blue', label = 'Training')
    plt.plot(history["val_loss"], color = 'orange', label = 'Testing')
    plt.xlabel("Epochs")
    plt.annotate('Max:' + str(round(ymax1,2)) , xy = (xmax1, ymax1), xytext = (xmax1*0.93, 1.07*ymax1), 
                    arrowprops=dict(facecolor='blue', headwidth= 6, headlength =9))
    plt.annotate('Max:' + str(round(ymax2,2)) , xy = (xmax2, ymax2), xytext = (xmax2*0.93, 1.07*ymax2), 
                    arrowprops=dict(facecolor='goldenrod', headwidth= 6, headlength =9))
    plt.legend()
    plt.show()
    # Graph accuracy
    ymax3 = max(history["acc"])
    xmax3 = history["acc"].index(ymax3)
    ymax4 = max(history["val_acc"])
    xmax4 = history["val_acc"].index(ymax4)
    ymax5 = max(history["top5-acc"])
    xmax5 = history["top5-acc"].index(ymax5)
    ymax6 = max(history["val_top5-acc"])
    xmax6 = history["val_top5-acc"].index(ymax6)
    plt.subplot(2,1,1)
    plt.title('Classification accuracy')
    plt.plot(history['acc'], color = 'blue', label = 'Training')
    plt.plot(history['val_acc'], color = 'orange', label = 'Testing')
    plt.annotate('Max:' + str(round(ymax3,2)) , xy = (xmax3, ymax3), xytext = (xmax3*0.93, 1.2*ymax3), 
                    arrowprops=dict(facecolor='blue', headwidth= 6, headlength =9))
    plt.annotate('Max:' + str(round(ymax4,2)) , xy = (xmax4, ymax4), xytext = (xmax4*0.93, 0.7*ymax4), 
                    arrowprops=dict(facecolor='goldenrod', headwidth= 6, headlength =9))
    plt.subplot(2,1,2)
    plt.title('Classification top5-acc')
    plt.plot(history['top5-acc'], color = 'blue', label = 'Training')
    plt.plot(history['val_top5-acc'], color = 'orange', label = 'Testing')
    plt.annotate('Max:' + str(round(ymax5,2)) , xy = (xmax5, ymax5), xytext = (xmax5*0.93, 1.2*ymax5), 
                    arrowprops=dict(facecolor='blue', headwidth= 6, headlength =9))
    plt.annotate('Max:' + str(round(ymax6,2)) , xy = (xmax6, ymax6), xytext = (xmax6*0.87, 1.2*ymax6), 
                    arrowprops=dict(facecolor='goldenrod', headwidth= 6, headlength =9))
    plt.xlabel("Epochs")
    plt.suptitle("Learning Curves")
    plt.legend()
    plt.tight_layout()
    plt.show()

## Obtain activations + Visualization

In [10]:
# Preprocessing Layers + Patches + One dense layer
def Preprocessing(num_example):
    augmented = data_augmentation(x_train[num_example])
    b = Patches(patch_size, num_patches)(augmented)
    a = layers.Dense(units=embedding_dim)(b)
    inp = tf.reshape(a,[1,embedding_dim,num_patches])
    return inp

In [11]:
# Creates a random vector with indexes of a random batch selection and also regularizes the selected batch
def Batch_Preprocessing(batch_size):
    #Vector with the number of Sample of the Xtrain
    a  = list(range(0,x_train.shape[0]))
    b = random.sample(a,batch_size)
    batch_regularization = list()
    for i in range(0,batch_size):
        inter_result = Preprocessing(b[i])
        batch_regularization.append(inter_result)
    return batch_regularization


In [12]:
def visualize_out(result,layer_number,example):
    fig, (ax1, ax2)= plt.subplots(1,2)
    ax1.imshow(x_train[example])
    ax1.set_title('Original_Figure, Class: #' + str(y_train[example][0]))
    ax2.imshow(result[layer_number])
    ax2.set_title('Activations of MLP block of the Mixer #: '+ '"' + str(layer_number) + '"')
    return None

In [13]:
def Mixer_Layer_Outputs(model_input, model_output,example):
    #The input is fixed to the beginning of the mlp blocks
    intermediate_model=tf.keras.models.Model(inputs=model_input.input,outputs=model_output.output)
    #This reshape is necessary for the input of the model
    example = tf.reshape(example,[1,num_patches,embedding_dim])
    #Inference
    intermediate_prediction =intermediate_model.predict(example)
    #This reshape is standardize the output
    layactivation = intermediate_prediction.reshape((embedding_dim,num_patches))
    return layactivation

In [14]:
## Computes the outputs of each MLP-mixer Layer
def Mixer_Activations(model, example):
    total_activations = list()
    for i in range(num_blocks):
        model_input = model.layers[4].layers[0]
        model_output = model.layers[4].layers[i]
        int_total_activations = Mixer_Layer_Outputs(model_input, model_output, example)
        total_activations.append(int_total_activations)
    return  total_activations

In [15]:
#Average of layer's activation
def Prom_Mixer_Activations_Blocks(model,batch_regularization):
    sum = list()
    for i in range(0,num_blocks):
        sum_raw = np.zeros((embedding_dim,num_patches))
        sum.append(sum_raw)
    for i in range(0,batch_size):
        mixer_raw = Mixer_Activations(model,batch_regularization[i])
        for i in range(0,num_blocks):
            sum[i] = np.add(mixer_raw[i],sum[i])
    prom_mixer_activations = [ (number / batch_size)  for number in sum]
    return prom_mixer_activations

# CKA

In [16]:
# Calculates a heatmap according to the selection of a CKA_Kernel (preferred) or CKA_Linear
def Heatmap(result,type,sigma):
    dim = len(result)
    k = (dim - 1)
    heatmap_CKA = np.zeros((dim,dim))
    for i in range(0,dim):
        tr = (dim - 1)
        for j in range(0,dim):
            if type == 'kernel':
                heatmap_CKA[k][tr] = cka(gram_rbf(result[i],sigma),gram_rbf(result[j],sigma))
            elif type == 'linear':
                heatmap_CKA[k][tr] = cka(gram_linear(result[i]),gram_linear(result[j])) 
            else:
                print('There is no such category, try again')
                break

            tr -= 1
        k -= 1
    #print('CKA' + type + 'calculated')
    return heatmap_CKA

In [17]:
#Average of heatmaps (obsolet)
def Prom_Mixer_Heatmaps(batch_result,type):
    mat_heatmaps = list()
    prom_mixer_heatmap_raw = np.zeros((num_blocks,num_blocks))
    for i in range(0,batch_size):
        mixer_activations_raw = Mixer_Activations(batch_result[i])
        heatmap_raw = Heatmap(mixer_activations_raw, type)
        mat_heatmaps.append(heatmap_raw)
        prom_mixer_heatmap_raw = np.add(heatmap_raw,prom_mixer_heatmap_raw)
    prom_mixer_heatmap =  prom_mixer_heatmap_raw/batch_size  
    return prom_mixer_heatmap,mat_heatmaps

In [18]:
def visualize_Heatmap(heatmap,type,bl):
    #Number of thats that you want to appear in the plot
    tri = 4
    if type == 'kernel' or type == 'linear':
        dim = len(heatmap)
        axis_labels = list()
        for i in range(0,dim):
            axis_labels_inter = str('%i'%(i+1))
            axis_labels.append(axis_labels_inter)
        _, ax = plt.subplots(figsize=(3,3))
        ax = sns.heatmap(heatmap, xticklabels=axis_labels[::-1], yticklabels=axis_labels[::-1], ax = ax, annot=bl)
        #sns.heatmap(heatmap, xticklabels=2, yticklabels=2, ax = ax, annot=bl, cbar=True)   
        ax.invert_xaxis()
        ax.axhline(y = 0, color='k',linewidth = 4)
        ax.axhline(y = heatmap.shape[1], color = 'k', linewidth = 4)
        ax.axvline(x = 0, color ='k',linewidth = 4)
        ax.axvline(x = heatmap.shape[0], color = 'k', linewidth = 4)

        ax.set_title("CKA-"+ type)   
        ax.set_xlabel("Layer")
        ax.set_ylabel("Layer")
        plt.yticks(rotation=0)
        plt.locator_params(axis='x',nbins=tri)
        plt.locator_params(axis='y',nbins=tri)
        plt.savefig('CKA_'+ type +'.png', dpi=300)
        
    else:
        print('There is no such category, try again')

# Experiment 1 : Understand Network Architecture

## 1A: Different Depths

In [19]:
#Create different mlpmixers according to an array of widths or depths
def mlpmixer_iterations(num_patches,experiment,embedding_dim,num_blocks):
    it_widths = len(embedding_dim)
    it_blocks = len(num_blocks)
    for j in range(it_widths):
        for i in range(it_blocks):
            mlpmixer_blocks = keras.Sequential(
            [MLPMixerLayer(num_patches, embedding_dim[j], dropout_rate) for _ in range(num_blocks[i])] # creates the number of block without a 
            )
            mlpmixer_classifier = build_classifier(mlpmixer_blocks,embedding_dim[j]) # Returns the model
            history,accuracy, top_5_accuracy = run_experiment(mlpmixer_classifier)
            #Saving Results
            pwd = 'Results_Article/'+ str(experiment) +'/mlpmixer_'+ str(num_blocks[i]) + 'ly_' + str(embedding_dim[j]) + 'Dc'
            mlpmixer_classifier.save(pwd)
            np.save( pwd + '/history.npy',history.history)
            with open(pwd + '/accuracy.pkl','wb') as file:
                pickle.dump(accuracy,file)
            with open(pwd + '/top5-accuracy.pkl','wb') as file:
                pickle.dump(top_5_accuracy,file)

Batch

In [20]:
# Heatmap of Average of layer's activation
sigma = 1
type = 'kernel'
embedding_dim = 384
blocks_total = num_blocks
######################################################

In [21]:
#Run separtely once to avoid randomness 
batch_prepro = Batch_Preprocessing(batch_size)

# 1B

## Configure the hyperparameters

In [22]:
# This is the only parameter that have to be initialized since, all the parameters are shared with Experiment 1A
embedding_d = embedding_dim  # Fixed Embedding Dimension from experiment 1A
path_1B = 'Results_Article/1B'

In [23]:
def evol_accuracy(all_models,num_blocks):
    total_plots=list()
    for f in range(len(all_models)) :
        testing_model = all_models[f]
        partial_plots = list()
        for j in range(num_blocks[f]):
            #Define the Mixer Block that are going to participate (Cumulative Approach)
            inter_input = testing_model.layers[4].layers[0].input
            inter_output = testing_model.layers[4].layers[j].output
            partial_models=tf.keras.models.Model(inputs=inter_input,outputs=inter_output, name = 'Mixer_Blocks')
            #Create the structure of the model
            inputs = layers.Input(shape=input_shape)
            augmented = data_augmentation(inputs)
            patches = Patches(patch_size, num_patches)(augmented)
            x = testing_model.layers[3](patches)
            intermediate_output  =  partial_models(x)
            representation = layers.GlobalAveragePooling1D()(intermediate_output)
            output =  layers.Dense(units=num_classes, activation='softmax')(representation) # Linear regression that is going to be trained
            final_modelx =   keras.Model(inputs=inputs, outputs=output)
            #Set the condition to not trainable
            final_modelx.layers[3].trainable = False
            final_modelx.layers[4].trainable = False
            __,accuracy,__= run_experiment(final_modelx)
            with open(path_1B + '/accuracy_Blocks_'+ str(num_blocks[f]) + '_L' + str(j+1)+ '.pkl','wb') as file:
                        pickle.dump(accuracy,file)
            partial_plots.append(accuracy)
        total_plots.append(partial_plots)
    return total_plots

Change the path in this cell (loading results from the experiment 1A)

In [24]:
num_blocks = [8, 12, 24]
listnumblocks = num_blocks

In [25]:
all_models = list()
for layer in listnumblocks:
    #Call the folder
    pwd1 = 'Results_Article/1A/mlpmixer_'+ str(layer) + 'ly_' + str(embedding_d) + 'Dc' 
    layers_models = tf.keras.models.load_model(pwd1, compile=False)
    all_models.append(layers_models)      

In [26]:
tested_acc_evolution = evol_accuracy(all_models,listnumblocks)

Epoch 1/50


C:\Users\alach\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


88/88 [==============================] - 3s 27ms/step - loss: 2.4764 - acc: 0.2906 - top5-acc: 0.8110 - val_loss: 1.8434 - val_acc: 0.3594 - val_top5-acc: 0.8614 - lr: 0.0050
Epoch 2/50
88/88 [==============================] - 2s 24ms/step - loss: 1.9435 - acc: 0.3278 - top5-acc: 0.8459 - val_loss: 1.8080 - val_acc: 0.3710 - val_top5-acc: 0.8678 - lr: 0.0050
Epoch 3/50
88/88 [==============================] - 2s 24ms/step - loss: 1.8426 - acc: 0.3548 - top5-acc: 0.8581 - val_loss: 1.7068 - val_acc: 0.3912 - val_top5-acc: 0.8874 - lr: 0.0050
Epoch 4/50
88/88 [==============================] - 2s 24ms/step - loss: 1.8235 - acc: 0.3582 - top5-acc: 0.8627 - val_loss: 1.6956 - val_acc: 0.3966 - val_top5-acc: 0.8788 - lr: 0.0050
Epoch 5/50
88/88 [==============================] - 2s 24ms/step - loss: 1.8417 - acc: 0.3596 - top5-acc: 0.8629 - val_loss: 1.7472 - val_acc: 0.3702 - val_top5-acc: 0.8848 - lr: 0.0050
Epoch 6/50
88/88 [==============================] - 2s 25ms/step - loss: 1.8047 -

Epoch 41/50
88/88 [==============================] - 2s 25ms/step - loss: 1.5910 - acc: 0.4334 - top5-acc: 0.8967 - val_loss: 1.5201 - val_acc: 0.4488 - val_top5-acc: 0.9104 - lr: 3.1250e-04
Epoch 42/50
88/88 [==============================] - 2s 24ms/step - loss: 1.5845 - acc: 0.4346 - top5-acc: 0.8966 - val_loss: 1.5203 - val_acc: 0.4598 - val_top5-acc: 0.9076 - lr: 3.1250e-04
Epoch 43/50
88/88 [==============================] - 2s 24ms/step - loss: 1.5902 - acc: 0.4327 - top5-acc: 0.8971 - val_loss: 1.5339 - val_acc: 0.4426 - val_top5-acc: 0.9092 - lr: 3.1250e-04
Epoch 44/50
88/88 [==============================] - 2s 24ms/step - loss: 1.5898 - acc: 0.4327 - top5-acc: 0.8944 - val_loss: 1.5217 - val_acc: 0.4556 - val_top5-acc: 0.9110 - lr: 3.1250e-04
Epoch 45/50
88/88 [==============================] - 2s 24ms/step - loss: 1.5832 - acc: 0.4362 - top5-acc: 0.8968 - val_loss: 1.5298 - val_acc: 0.4422 - val_top5-acc: 0.9092 - lr: 3.1250e-04
Epoch 46/50
88/88 [==========================

88/88 [==============================] - 3s 33ms/step - loss: 1.4488 - acc: 0.4829 - top5-acc: 0.9224 - val_loss: 1.3411 - val_acc: 0.5126 - val_top5-acc: 0.9410 - lr: 0.0025
Epoch 27/50
88/88 [==============================] - 3s 33ms/step - loss: 1.4614 - acc: 0.4782 - top5-acc: 0.9226 - val_loss: 1.3766 - val_acc: 0.5064 - val_top5-acc: 0.9330 - lr: 0.0025
Epoch 28/50
88/88 [==============================] - 3s 33ms/step - loss: 1.4229 - acc: 0.4902 - top5-acc: 0.9270 - val_loss: 1.3080 - val_acc: 0.5300 - val_top5-acc: 0.9412 - lr: 0.0012
Epoch 29/50
88/88 [==============================] - 3s 33ms/step - loss: 1.4151 - acc: 0.4905 - top5-acc: 0.9277 - val_loss: 1.3386 - val_acc: 0.5150 - val_top5-acc: 0.9380 - lr: 0.0012
Epoch 30/50
88/88 [==============================] - 3s 33ms/step - loss: 1.4285 - acc: 0.4892 - top5-acc: 0.9261 - val_loss: 1.3069 - val_acc: 0.5312 - val_top5-acc: 0.9436 - lr: 0.0012
Epoch 31/50
88/88 [==============================] - 3s 33ms/step - loss: 1.4

...

Epoch 23/50
88/88 [==============================] - 16s 180ms/step - loss: 1.2236 - acc: 0.5644 - top5-acc: 0.9488 - val_loss: 1.1516 - val_acc: 0.5946 - val_top5-acc: 0.9598 - lr: 0.0025
Epoch 24/50
88/88 [==============================] - 16s 181ms/step - loss: 1.2210 - acc: 0.5660 - top5-acc: 0.9493 - val_loss: 1.1507 - val_acc: 0.5966 - val_top5-acc: 0.9594 - lr: 0.0025
Epoch 25/50
88/88 [==============================] - 16s 181ms/step - loss: 1.2228 - acc: 0.5647 - top5-acc: 0.9480 - val_loss: 1.1425 - val_acc: 0.5952 - val_top5-acc: 0.9600 - lr: 0.0025
Epoch 26/50
88/88 [==============================] - 16s 182ms/step - loss: 1.2226 - acc: 0.5650 - top5-acc: 0.9490 - val_loss: 1.1518 - val_acc: 0.5968 - val_top5-acc: 0.9590 - lr: 0.0025
Epoch 27/50
88/88 [==============================] - 16s 181ms/step - loss: 1.2222 - acc: 0.5630 - top5-acc: 0.9485 - val_loss: 1.1383 - val_acc: 0.5962 - val_top5-acc: 0.9580 - lr: 0.0025
Epoch 28/50
88/88 [==============================] - 16

88/88 [==============================] - 17s 193ms/step - loss: 1.2144 - acc: 0.5652 - top5-acc: 0.9515 - val_loss: 1.1328 - val_acc: 0.6070 - val_top5-acc: 0.9592 - lr: 0.0050
Epoch 9/50
88/88 [==============================] - 17s 193ms/step - loss: 1.2080 - acc: 0.5676 - top5-acc: 0.9519 - val_loss: 1.1683 - val_acc: 0.5890 - val_top5-acc: 0.9566 - lr: 0.0050
Epoch 10/50
88/88 [==============================] - 17s 193ms/step - loss: 1.2067 - acc: 0.5680 - top5-acc: 0.9513 - val_loss: 1.1312 - val_acc: 0.6024 - val_top5-acc: 0.9604 - lr: 0.0050
Epoch 11/50
88/88 [==============================] - 17s 194ms/step - loss: 1.2027 - acc: 0.5712 - top5-acc: 0.9504 - val_loss: 1.1187 - val_acc: 0.6060 - val_top5-acc: 0.9636 - lr: 0.0050
Epoch 12/50
88/88 [==============================] - 17s 194ms/step - loss: 1.2028 - acc: 0.5702 - top5-acc: 0.9517 - val_loss: 1.1499 - val_acc: 0.5862 - val_top5-acc: 0.9600 - lr: 0.0050
Epoch 13/50
88/88 [==============================] - 17s 193ms/step 

313/313 [==============================] - 28s 91ms/step - loss: 1.1144 - acc: 0.6051 - top5-acc: 0.9594
Test accuracy: 60.51%
Test top 5 accuracy: 95.94%
Epoch 1/50
88/88 [==============================] - 22s 214ms/step - loss: 1.4319 - acc: 0.4969 - top5-acc: 0.9193 - val_loss: 1.1647 - val_acc: 0.5848 - val_top5-acc: 0.9580 - lr: 0.0050
Epoch 2/50
88/88 [==============================] - 18s 202ms/step - loss: 1.1967 - acc: 0.5751 - top5-acc: 0.9522 - val_loss: 1.1233 - val_acc: 0.6024 - val_top5-acc: 0.9612 - lr: 0.0050
Epoch 3/50
88/88 [==============================] - 18s 202ms/step - loss: 1.1664 - acc: 0.5827 - top5-acc: 0.9550 - val_loss: 1.0783 - val_acc: 0.6164 - val_top5-acc: 0.9628 - lr: 0.0050
Epoch 4/50
88/88 [==============================] - 18s 201ms/step - loss: 1.1432 - acc: 0.5928 - top5-acc: 0.9577 - val_loss: 1.0931 - val_acc: 0.6040 - val_top5-acc: 0.9640 - lr: 0.0050
Epoch 5/50
88/88 [==============================] - 18s 202ms/step - loss: 1.1374 - acc: 0.59

88/88 [==============================] - 18s 201ms/step - loss: 1.0817 - acc: 0.6159 - top5-acc: 0.9623 - val_loss: 1.0224 - val_acc: 0.6360 - val_top5-acc: 0.9664 - lr: 0.0012
Epoch 37/50
88/88 [==============================] - 18s 202ms/step - loss: 1.0742 - acc: 0.6219 - top5-acc: 0.9616 - val_loss: 1.0226 - val_acc: 0.6348 - val_top5-acc: 0.9660 - lr: 0.0012
Epoch 38/50
88/88 [==============================] - 18s 201ms/step - loss: 1.0764 - acc: 0.6200 - top5-acc: 0.9623 - val_loss: 1.0131 - val_acc: 0.6392 - val_top5-acc: 0.9658 - lr: 6.2500e-04
Epoch 39/50
88/88 [==============================] - 18s 203ms/step - loss: 1.0756 - acc: 0.6188 - top5-acc: 0.9620 - val_loss: 1.0199 - val_acc: 0.6400 - val_top5-acc: 0.9664 - lr: 6.2500e-04
Epoch 40/50
88/88 [==============================] - 18s 203ms/step - loss: 1.0762 - acc: 0.6184 - top5-acc: 0.9627 - val_loss: 1.0209 - val_acc: 0.6328 - val_top5-acc: 0.9666 - lr: 6.2500e-04
Epoch 41/50
88/88 [==============================] - 18

Epoch 21/50
88/88 [==============================] - 18s 209ms/step - loss: 0.9283 - acc: 0.6720 - top5-acc: 0.9735 - val_loss: 0.9256 - val_acc: 0.6768 - val_top5-acc: 0.9736 - lr: 0.0012
Epoch 22/50
88/88 [==============================] - 18s 209ms/step - loss: 0.9311 - acc: 0.6714 - top5-acc: 0.9730 - val_loss: 0.9207 - val_acc: 0.6720 - val_top5-acc: 0.9726 - lr: 0.0012
Epoch 23/50
88/88 [==============================] - 18s 209ms/step - loss: 0.9301 - acc: 0.6738 - top5-acc: 0.9729 - val_loss: 0.9178 - val_acc: 0.6782 - val_top5-acc: 0.9722 - lr: 0.0012
Epoch 24/50
88/88 [==============================] - 18s 209ms/step - loss: 0.9276 - acc: 0.6734 - top5-acc: 0.9733 - val_loss: 0.9229 - val_acc: 0.6720 - val_top5-acc: 0.9742 - lr: 0.0012
Epoch 25/50
88/88 [==============================] - 18s 209ms/step - loss: 0.9266 - acc: 0.6724 - top5-acc: 0.9734 - val_loss: 0.9193 - val_acc: 0.6772 - val_top5-acc: 0.9736 - lr: 0.0012
Epoch 26/50
88/88 [==============================] - 18

In [27]:
##Ignore the folowwing lines

In [28]:
# # Calculates a heatmap according to the selection of a CKA_Kernel (preferred) or CKA_Linear
# def Heatmap_2(result1,result2,type,sigma):
#     dim1 = len(result1)
#     dim2 = len(result2)
#     k = (dim1 - 1)
#     heatmap_CKA = np.zeros((dim1,dim2))
#     for i in range(0,dim1):
#         tr = (dim2 - 1)
#         for j in range(0,dim2):
#             if type == 'kernel':
#                 heatmap_CKA[k][tr] = cka(gram_rbf(result1[i],sigma),gram_rbf(result2[j],sigma))
#             elif type == 'linear':
#                 heatmap_CKA[k][tr] = cka(gram_linear(result1[i]),gram_linear(result2[j])) 
#             else:
#                 print('There is no such category, try again')
#                 break

#             tr -= 1
#         k -= 1
#     #print('CKA' + type + 'calculated')
#     return heatmap_CKA

In [29]:
# def visualize_Heatmap_mul(heatmap,type,bl):
#     #Number of thats that you want to appear in the plot
#     tri = 4
#     if type == 'kernel' or type == 'linear':
#         dim1 = heatmap.shape[1]
#         dim2 = heatmap.shape[0]
#         axis_labelsx = list()
#         axis_labelsy = list()
#         for i in range(0,dim1):
#             axis_labels_inter = str('%i'%(i+1))
#             axis_labelsx.append(axis_labels_inter)
    
#         for i in range(0,dim2):
#             axis_labels_inter = str('%i'%(i+1))
#             axis_labelsy.append(axis_labels_inter)
        
#         _, ax = plt.subplots(figsize=(7,3))
#         ax = sns.heatmap(heatmap, xticklabels=axis_labelsx[::-1], yticklabels=axis_labelsy[::-1], ax = ax, annot=bl)
#         #sns.heatmap(heatmap, xticklabels=2, yticklabels=2, ax = ax, annot=bl, cbar=True)   
#         ax.invert_xaxis()
        
#         # ax.axhline(y = 0, color='k',linewidth = 4)
#         # ax.axhline(y = heatmap.shape[1], color = 'k', linewidth = 4)
#         # ax.axvline(x = 0, color ='k',linewidth = 4)
#         # ax.axvline(x = heatmap.shape[0], color = 'k', linewidth = 4)

#         ax.set_title("CKA-"+ type)   
#         ax.set_xlabel("Layer")
#         ax.set_ylabel("Layer")
#         plt.yticks(rotation=0)
#         plt.locator_params(axis='x',nbins=tri)
#         plt.locator_params(axis='y',nbins=tri)
#         plt.savefig('CKA_'+ type +'.png', dpi=300)
        
#     else:
#         print('There is no such category, try again')

In [30]:
# val1 = 8
# val2 = 12
# path = 'Results_Article/1A/mlpmixer_'+ str(val1) +'ly_384Dc'
# with open(path + '/activations_'+ str(val1) +'ly_384Dc.pkl','rb') as file:
#     tested_activations1 = pickle.load(file)
#     path = 'Results_Article/1A/mlpmixer_'+ str(val2) +'ly_384Dc'
# with open(path + '/activations_'+ str(val2) +'ly_384Dc.pkl','rb') as file:
#     tested_activations2 = pickle.load(file)

In [31]:
# testing = Heatmap_2(tested_activations1, tested_activations2, 'kernel',1)

In [32]:
# visualize_Heatmap_mul(testing,'kernel',False)